# 第05章 — 物理ライブラリ

- 物理エンジン（Box2D, Matter.jsなど）は衝突判定・積分・拘束を管理する。
- ボディ（円/多角形）、フィクスチャ（材質）、ジョイント（ばね・ロッド）を組み合わせて扱う。
- 軽いアダプタを挟んで、クリエイティブ側のコードを表現的に保つとよい。
- スケールと単位系は重要で、ピクセルとメートルの対応を一貫させる。

試してみよう: 本物のエンジンに差し替えてもインタフェースが似るようアダプタを保つ。

In [1]:
# 依存なしで物理アダプタの形を示すスタブを用意
from dataclasses import dataclass

class Vector:
    def __init__(self, x=0.0, y=0.0):
        self.x, self.y = float(x), float(y)

    def __add__(self, other):
        return Vector(self.x + other.x, self.y + other.y)

    def __mul__(self, k):
        return Vector(self.x * k, self.y * k)

    __rmul__ = __mul__

    def __repr__(self):
        return f"Vector({self.x:.2f}, {self.y:.2f})"

@dataclass
class BodyConfig:
    position: tuple
    shape: str = "circle"
    radius: float = 1.0
    density: float = 1.0
    friction: float = 0.3


In [2]:
# FakePhysicsWorldの簡易実装
class FakePhysicsWorld:
    def __init__(self, gravity=(0, -9.8)):
        self.gravity = gravity
        self.bodies = []

    def create_body(self, cfg: BodyConfig):
        body = {"cfg": cfg, "vel": Vector(0, 0)}
        self.bodies.append(body)
        return body

    def step(self, dt=1/60):
        for body in self.bodies:
            gx, gy = self.gravity
            body["vel"] += Vector(gx * dt, gy * dt)


In [3]:
# スタブワールドでボディを生成して重力で更新
world = FakePhysicsWorld()
ball = world.create_body(BodyConfig(position=(0, 5), radius=0.5))

for i in range(3):
    world.step()
    print(f"step {i}: vel={ball['vel']}")


step 0: vel=Vector(0.00, -0.16)
step 1: vel=Vector(0.00, -0.33)
step 2: vel=Vector(0.00, -0.49)


### 追加例: 単純なワールドで複数ボディを積分

In [4]:
# 最低限のワールドとボディを使った積分サンプル
class SimpleBody:
    def __init__(self, x, y, mass=1.0):
        self.x, self.y = x, y
        self.vx = 0.0
        self.vy = 0.0
        self.mass = mass

class SimpleWorld:
    def __init__(self, gravity=(0, -9.8)):
        self.gx, self.gy = gravity
        self.bodies = []

    def add_body(self, body):
        self.bodies.append(body)

    def step(self, dt=0.1):
        for b in self.bodies:
            b.vx += self.gx * dt
            b.vy += self.gy * dt
            b.x += b.vx * dt
            b.y += b.vy * dt


In [5]:
# 複数ボディを追加して位置の変化を記録
world = SimpleWorld()
world.add_body(SimpleBody(0, 5, mass=1))
world.add_body(SimpleBody(2, 8, mass=2))

for i in range(5):
    world.step()
    positions = [(round(b.x, 2), round(b.y, 2)) for b in world.bodies]
    print(f"step {i}: {positions}")


step 0: [(0.0, 4.9), (2.0, 7.9)]
step 1: [(0.0, 4.71), (2.0, 7.71)]
step 2: [(0.0, 4.41), (2.0, 7.41)]
step 3: [(0.0, 4.02), (2.0, 7.02)]
step 4: [(0.0, 3.53), (2.0, 6.53)]
